In [1]:
#Make the ring Q(i*sqrt(7))

K = QuadraticField(-7,'a')
B = K.integral_basis()

#Here are some useful numbers.  a=i*sqrt(7).
a = B[1]
tau = (1+a)/2

#################################################
#Here are all the points and matrices:
Id = matrix([[1,0,0],[0,1,0],[0,0,1]])

Pinf = matrix([[1],[0],[0]])

P0 = matrix([[0],[0],[1]])
I0 = matrix([[0,0,1],[0,-1,0],[1,0,0]])

P21 = matrix([[-1/4+3*a/4],[tau/2],[1]])
A21 = matrix([[-1/2*a-5/2,-5,5],[-1,-1,0],[1/2*a-1/2,a,-a]])

P22 = matrix([[tau-1],[(tau+1)/2],[1]])
#SEE BELOW A42

P41 = matrix([[a/2],[0],[1]])
A41 = matrix([[a,0,4],[0,1,0],[2,0,-a]])

P42 = matrix([[(tau-1)/2],[tau/2],[1]])
A42 = matrix([[tau-1,tau,2],[tau,2,1-tau],[2,1-tau,-tau]])

A22 = A42^(-1)*I0

P43 = matrix([[-1/2],[(tau+1)/2],[1]])
A43 = matrix([[-1,tau-2,2],[tau+1,3,-tau-1],[2,2-tau,-1]])

P44 = matrix([[(5*tau-3)/4],[(tau+1)/4],[1]])
#SEE BELOW Gamma_infty

P45 = matrix([[(3*tau-2)/2],[(tau+2)/4],[1]])
#SEE BELOW Gamma_infty

P71 = matrix([[-1/a],[0],[1]])
A71 = matrix([[-1,0,0],[0,1,0],[a,0,-1]])

P72 = matrix([[-2/a],[0],[1]])
A72 = matrix([[-2,0,a],[0,1,0],[a,0,3]])

P73 = matrix([[-3/a],[0],[1]])
A73 = matrix([[-3,0,a],[0,1,0],[a,0,2]])

P74 = matrix([[-4/a],[0],[1]])
A74 = matrix([[-4,0,-a],[0,1,0],[a,0,-2]])

P75 = matrix([[-5/a],[0],[1]])
A75 = matrix([[-5,0,3*a],[0,1,0],[a,0,4]])

P76 = matrix([[-6/a],[0],[1]])
A76 = matrix([[-6,0,a],[0,1,0],[a,0,1]])

#Gamma_infty
Tv = matrix([[1,0,a],[0,1,0],[0,0,1]])
T1 = matrix([[1,-1,(-1+a)/2],[0,1,1],[0,0,1]])
Ttau = matrix([[1,-(1-a)/2,-1],[0,1,(1+a)/2],[0,0,1]])
R = matrix([[1,0,0],[0,-1,0],[0,0,1]])
matrices_gamma_infinity = [['R',R],['T1',T1],['T1^-1',T1^-1],['Ttau',Ttau],['Ttau^-1',Ttau^-1],['Tv',Tv],['Tv^-1',Tv^-1]]

A44 = A75^(-1)*T1^(-1)*Tv*I0
A45 = A75^(-1)*A41^(-1)*A42


# Each matrix is stored as a 2-tuple [name, matrix]
matrices = [['I0',I0],['A21',A21],['A22',A22],['A41',A41],['A42',A42],['A43',A43],['A44',A44],['A45',A45],['A71',A71],['A72',A72],['A73',A73],['A74',A74],['A75',A75],['A76',A76]]

#each point is stored as a 4-tuple [name, vector, depth, denominator-clearer]
points = [['P0',P0,1,1],['P21',P21,2,tau-1],['P22',P22,2,tau],['P41',P41,4,2],['P42',P42,4,2],['P43',P43,4,2],['P44',P44,4,tau-2],['P45',P45,4,tau+1],['P71',P71,7,a],['P72',P72,7,a],['P73',P73,7,a],['P74',P74,7,a],['P75',P75,7,a],['P76',P76,7,a]]

In [2]:
# these are the scalars who clear the denominators of the prism points of each of their respective depths.  Technically d1 should contain 1, not empty, but that would be silly and would just mean we mutiply by 1 a lot.
d1 = []
d2 = [tau, tau-1]
d4 = [2,tau+1,tau-2]
d7 = [a]
depths = [d2,d4,d7]

In [3]:
#Compute the inner product of two vectors
def ip(v1,v2):
    return v2[2].conjugate()*v1[0] + v2[1].conjugate()*v1[1] + v2[0].conjugate()*v1[2]

#Test if a (rational) number (not a vector) is in the ring of integers.
def IsIntegral(n):
    if (K.coerce(n)[0] in ZZ) & (K.coerce(n)[1] in ZZ):
        return True
    else:
        if (K.coerce(n)[0]+1/2 in ZZ) & (K.coerce(n)[1]+1/2 in ZZ):
            return True
        else:
            return False

#Given a vector v, find its primitive integral lift.  Only works on points of depth up to 7 because what it does is multiply by all possible scalars that could clear the denominator and then checks if the coordinates are integral.
def PrimIntLift(v):
    if IsIntegral(v[0][0]) & IsIntegral(v[1][0]) & IsIntegral(v[2][0]):
        return v
    for d in depths:
        for q in d:
            w = v*q
            if IsIntegral(w[0][0]) & IsIntegral(w[1][0]) & IsIntegral(w[2][0]):
                return w
    print("depth greater than 7")
    return False
   
#Find the level between two points.  Only works on points of depth up to 7.
def level(v1,v2):
    return K.coerce(ip(PrimIntLift(v1),PrimIntLift(v2))).norm()

#Find the depth of a point.  Only works on points of depth up to 7.
def depth(v):
    return K.coerce(PrimIntLift(v)[2][0]).norm()


In [4]:
#How far away is the furthest point from the origin in the prism?
def vectCygan(v1,v2):
    nv1 = v1*v1[2][0]^(-1)
    nv2 = v2*v2[2][0]^(-1)
    prod = 2*ip(nv1,nv2)
    return (prod.norm())^(1/4)

max = [0,'P0']
for p in points:
    d = vectCygan(P0,p[1])
    if d > max[0]:
        max = [d,p[0]]
print(max[0])
print(float(max[0]))
print(max[1])

2*(9/7)^(1/4)
2.12968863361
P76


In [5]:
#so, the radius within which the center must lie in order for there to be a hope of an intersection is 
print(2*sqrt(2)+2*(9/7)^(1/4))
print(float(2*sqrt(2)+2*(9/7)^(1/4)))

#Thus, we draw a Cygan sphere of radius 5 centered at the origin, and more than cover it with prism translates, just to be safe.

2*sqrt(2) + 2*(9/7)^(1/4)
4.95811575835


In [6]:
#some helpful pictures to figure out the covering
var('x')
Q = plot(sqrt(25-x^2),(x,-5,5),xmin=-5.5,xmax=5.5,ymin=-5.5,ymax=5.5,aspect_ratio=1,axes=True)
Q += plot(-sqrt(25-x^2),(x,-5,5))
n = -7
while n < 7:
    Q += plot(sqrt(7)*x+n*sqrt(7),(x,-5.5,5.5),color='purple')
    Q += plot(-sqrt(7)*x+n*sqrt(7),(x,-5.5,5.5),color='purple')
    n += 1

n = -6
while n < 7:
    Q += plot(n*sqrt(7)/2,(x,-5.5,5.5),color='purple')
    n += 1

Q

In [7]:
S = plot(sqrt(5^4-x^4),(x,-5,5),xmin=-5.5,xmax=5.5,ymin=-26,ymax=26,aspect_ratio=.3,axes=False)
S += plot(-sqrt(5^4-x^4),(x,-5,5))


n = -5
while n < 6:
    S += plot(2*n*sqrt(7),(x,-4,-3),color='purple')
    S += plot(2*n*sqrt(7),(x,-2,-1),color='purple')
    S += plot(2*n*sqrt(7),(x,0,1),color='purple')
    S += plot(2*n*sqrt(7),(x,2,3),color='purple')
    S += plot(2*n*sqrt(7),(x,4,5),color='purple')
    S += plot((2*n+1)*sqrt(7),(x,-5,-4),color='purple')
    S += plot((2*n+1)*sqrt(7),(x,-3,-2),color='purple')
    S += plot((2*n+1)*sqrt(7),(x,-1,0),color='purple')
    S += plot((2*n+1)*sqrt(7),(x,1,2),color='purple')
    S += plot((2*n+1)*sqrt(7),(x,3,4),color='purple')
    S += line([(n,-27), (n,27)],color='purple')
    n += 1


S

In [8]:
# Here is a list of group elements, built by hand, whose translates cover the ball of radius 5 centered at the origin. (continues into the next cell.)

Blue0 = [['Id',Id],['Ttau',Ttau],['Ttau^2',Ttau^2],['Ttau^3',Ttau^3],
['Ttau^-1',Ttau^-1],['Ttau^-2',Ttau^-2],['Ttau^-3',Ttau^-3],['Ttau^-4',Ttau^-4],
['R',R],['Ttau*R',Ttau*R],['Ttau^2*R',Ttau^2*R],['Ttau^3*R',Ttau^3*R],['Ttau^4*R',Ttau^4*R],
['Ttau^-1*R',Ttau^-1*R],['Ttau^-2*R',Ttau^-2*R],['Ttau^-3*R',Ttau^-3*R]]

Red1 = [['T1',T1],['T1*Ttau',T1*Ttau],['T1*Ttau^2',T1*Ttau^2],['T1*Ttau^3',T1*Ttau^3],
['T1*Ttau^-1',T1*Ttau^-1],['T1*Ttau^-2',T1*Ttau^-2],['T1*Ttau^-3',T1*Ttau^-3],['T1*Ttau^-4',T1*Ttau^-4],
['T1*R',T1*R],['T1*Ttau*R',T1*Ttau*R],['T1*Ttau^2*R',T1*Ttau^2*R],['T1*Ttau^3*R',T1*Ttau^3*R],['T1*Ttau^4*R',T1*Ttau^4*R],
['T1*Ttau^-1*R',T1*Ttau^-1*R],['T1*Ttau^-2*R',T1*Ttau^-2*R],['T1*Ttau^-3*R',T1*Ttau^-3*R]]

Green2 = [['T1^2',T1^2],['T1^2*Ttau',T1^2*Ttau],['T1^2*Ttau^2',T1^2*Ttau^2],
['T1^2*Ttau^-1',T1^2*Ttau^-1],['T1^2*Ttau^-2',T1^2*Ttau^-2],['T1^2*Ttau^-3',T1^2*Ttau^-3],['T1^2*Ttau^-4',T1^2*Ttau^-4],
['T1^2*R',T1^2*R],['T1^2*Ttau*R',T1^2*Ttau*R],['T1^2*Ttau^2*R',T1^2*Ttau^2*R],['T1^2*Ttau^3*R',T1^2*Ttau^3*R],
['T1^2*Ttau^-1*R',T1^2*Ttau^-1*R],['T1^2*Ttau^-2*R',T1^2*Ttau^-2*R],['T1^2*Ttau^-3*R',T1^2*Ttau^-3*R]]

Pink3 = [['T1^3',T1^3],['T1^3*Ttau',T1^3*Ttau],['T1^3*Ttau^2',T1^3*Ttau^2],
['T1^3*Ttau^-1',T1^3*Ttau^-1],['T1^3*Ttau^-2',T1^3*Ttau^-2],['T1^3*Ttau^-3',T1^3*Ttau^-3],['T1^3*Ttau^-4',T1^3*Ttau^-4],
['T1^3*R',T1^3*R],['T1^3*Ttau*R',T1^3*Ttau*R],['T1^3*Ttau^2*R',T1^3*Ttau^2*R],['T1^3*Ttau^3*R',T1^3*Ttau^3*R],
['T1^3*Ttau^-1*R',T1^3*Ttau^-1*R],['T1^3*Ttau^-2*R',T1^3*Ttau^-2*R],['T1^3*Ttau^-3*R',T1^3*Ttau^-3*R]]

Orange4 = [['T1^4',T1^4],['T1^4*Ttau',T1^4*Ttau],
['T1^4*Ttau^-1',T1^4*Ttau^-1],['T1^4*Ttau^-2',T1^4*Ttau^-2],['T1^4*Ttau^-3',T1^4*Ttau^-3],['T1^4*Ttau^-4',T1^4*Ttau^-4],
['T1^4*R',T1^4*R],['T1^4*Ttau*R',T1^4*Ttau*R],['T1^4*Ttau^2*R',T1^4*Ttau^2*R],
['T1^4*Ttau^-1*R',T1^4*Ttau^-1*R],['T1^4*Ttau^-2*R',T1^4*Ttau^-2*R],['T1^4*Ttau^-3*R',T1^4*Ttau^-3*R]]

Blue5 = [['T1^5',T1^5],
['T1^5*Ttau^-1',T1^5*Ttau^-1],['T1^5*Ttau^-2',T1^5*Ttau^-2],['T1^5*Ttau^-3',T1^5*Ttau^-3],
['T1^5*R',T1^5*R],['T1^5*Ttau*R',T1^5*Ttau*R],
['T1^5*Ttau^-1*R',T1^5*Ttau^-1*R],['T1^5*Ttau^-2*R',T1^5*Ttau^-2*R],['T1^5*Ttau^-3*R',T1^5*Ttau^-3*R]]

Red6 = [['T1^6*R',T1^6*R],
['T1^6*Ttau^-1*R',T1^6*Ttau^-1*R],['T1^6*Ttau^-2*R',T1^6*Ttau^-2*R]]

Orangem1 = [['T1^-1',T1^-1],['T1^-1*Ttau',T1^-1*Ttau],['T1^-1*Ttau^2',T1^-1*Ttau^2],['T1^-1*Ttau^3',T1^-1*Ttau^3],
['T1^-1*Ttau^-1',T1^-1*Ttau^-1],['T1^-1*Ttau^-2',T1^-1*Ttau^-2],['T1^-1*Ttau^-3',T1^-1*Ttau^-3],['T1^-1*Ttau^-4',T1^-1*Ttau^-4],
['T1^-1*R',T1^-1*R],['T1^-1*Ttau*R',T1^-1*Ttau*R],['T1^-1*Ttau^2*R',T1^-1*Ttau^2*R],['T1^-1*Ttau^3*R',T1^-1*Ttau^3*R],['T1^-1*Ttau^4*R',T1^-1*Ttau^4*R],
['T1^-1*Ttau^-1*R',T1^-1*Ttau^-1*R],['T1^-1*Ttau^-2*R',T1^-1*Ttau^-2*R],['T1^-1*Ttau^-3*R',T1^-1*Ttau^-3*R]]

Pinkm2 = [['T1^-2',T1^-2],['T1^-2*Ttau',T1^-2*Ttau],['T1^-2*Ttau^2',T1^-2*Ttau^2],['T1^-2*Ttau^3',T1^-2*Ttau^3],
['T1^-2*Ttau^-1',T1^-2*Ttau^-1],['T1^-2*Ttau^-2',T1^-2*Ttau^-2],['T1^-2*Ttau^-3',T1^-2*Ttau^-3],
['T1^-2*R',T1^-2*R],['T1^-2*Ttau*R',T1^-2*Ttau*R],['T1^-2*Ttau^2*R',T1^-2*Ttau^2*R],['T1^-2*Ttau^3*R',T1^-2*Ttau^3*R],['T1^-2*Ttau^4*R',T1^-2*Ttau^4*R],
['T1^-2*Ttau^-1*R',T1^-2*Ttau^-1*R],['T1^-2*Ttau^-2*R',T1^-2*Ttau^-2*R]]

Greenm3 = [['T1^-3',T1^-3],['T1^-3*Ttau',T1^-3*Ttau],['T1^-3*Ttau^2',T1^-3*Ttau^2],['T1^-3*Ttau^3',T1^-3*Ttau^3],
['T1^-3*Ttau^-1',T1^-3*Ttau^-1],['T1^-3*Ttau^-2',T1^-3*Ttau^-2],['T1^-3*Ttau^-3',T1^-3*Ttau^-3],
['T1^-3*R',T1^-3*R],['T1^-3*Ttau*R',T1^-3*Ttau*R],['T1^-3*Ttau^2*R',T1^-3*Ttau^2*R],['T1^-3*Ttau^3*R',T1^-3*Ttau^3*R],['T1^-3*Ttau^4*R',T1^-3*Ttau^4*R],
['T1^-3*Ttau^-1*R',T1^-3*Ttau^-1*R],['T1^-3*Ttau^-2*R',T1^-3*Ttau^-2*R]]

Redm4 = [['T1^-4',T1^-4],['T1^-4*Ttau',T1^-4*Ttau],['T1^-4*Ttau^2',T1^-4*Ttau^2],['T1^-4*Ttau^3',T1^-4*Ttau^3],
['T1^-4*Ttau^-1',T1^-4*Ttau^-1],['T1^-4*Ttau^-2',T1^-4*Ttau^-2],
['T1^-4*R',T1^-4*R],['T1^-4*Ttau*R',T1^-4*Ttau*R],['T1^-4*Ttau^2*R',T1^-4*Ttau^2*R],['T1^-4*Ttau^3*R',T1^-4*Ttau^3*R],['T1^-4*Ttau^4*R',T1^-4*Ttau^4*R],
['T1^-4*Ttau^-1*R',T1^-4*Ttau^-1*R]]

Bluem5 = [['T1^-5',T1^-5],['T1^-5*Ttau',T1^-5*Ttau],['T1^-5*Ttau^2',T1^-5*Ttau^2],['T1^-5*Ttau^3',T1^-5*Ttau^3],
['T1^-5*Ttau^-1',T1^-5*Ttau^-1],
['T1^-5*R',T1^-5*R],['T1^-5*Ttau*R',T1^-5*Ttau*R],['T1^-5*Ttau^2*R',T1^-5*Ttau^2*R],['T1^-5*Ttau^3*R',T1^-5*Ttau^3*R]]

Orangem6 = [['T1^-6',T1^-6],['T1^-6*Ttau',T1^-6*Ttau],['T1^-6*Ttau^2',T1^-6*Ttau^2]]

In [9]:
TooMany = []

for g in Blue0:
    TooMany.append(g)
n=1
while n <=4:
    for g in Blue0:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-5:
    for g in Blue0:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1


for g in Red1:
    TooMany.append(g)
n=1
while n <=4:
    for g in Red1:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-6:
    for g in Red1:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1


for g in Green2:
    TooMany.append(g)
n=1
while n <=3:
    for g in Green2:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-6:
    for g in Green2:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1


for g in Pink3:
    TooMany.append(g)
n=1
while n <=3:
    for g in Pink3:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-7:
    for g in Pink3:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1


for g in Orange4:
    TooMany.append(g)
n=1
while n <=2:
    for g in Orange4:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-7:
    for g in Orange4:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1
    

for g in Blue5:
    TooMany.append(g)
n=1
while n <=2:
    for g in Blue5:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-8:
    for g in Blue5:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1


for g in Red6:
    TooMany.append(g)
n=1
while n <=1:
    for g in Red6:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-8:
    for g in Red6:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1


for g in Orangem1:
    TooMany.append(g)
n=1
while n <=5:
    for g in Orangem1:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-5:
    for g in Orangem1:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1



for g in Pinkm2:
    TooMany.append(g)
n=1
while n <=5:
    for g in Pinkm2:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-4:
    for g in Pinkm2:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1



for g in Greenm3:
    TooMany.append(g)
n=1
while n <=6:
    for g in Greenm3:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-4:
    for g in Greenm3:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1



for g in Redm4:
    TooMany.append(g)
n=1
while n <=6:
    for g in Redm4:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-3:
    for g in Redm4:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1



for g in Bluem5:
    TooMany.append(g)
n=1
while n <=7:
    for g in Bluem5:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-3:
    for g in Bluem5:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1



for g in Orangem6:
    TooMany.append(g)
n=1
while n <=6:
    for g in Orangem6:
        if n == 1:
            P = ['Tv'+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
        else:
            P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
            TooMany.append(P)
    n += 1
n=-1
while n >=-2:
    for g in Orangem6:
        P = ['Tv^'+str(n)+'*'+g[0],Tv^n*g[1]]
        TooMany.append(P)
    n -= 1

In [10]:
cycles = []
for g in TooMany:
    for P in points:
        for Q in points:
            gQ = g[1]*Q[1]
            l = level(P[1],gQ)
            if l <= 7:
                aa = points.index(P)
                c = points.index(Q)
                gam1Pb = matrices[aa][1]^(-1)*g[1]*matrices[c][1]*Pinf
                d = depth(gam1Pb)
                if d <=7:
                    C = [ matrices[aa], matrices[c], g]
                    cycles.append(C)

len(cycles)

399

In [11]:
#Spidercrawl around, outward from the identity, in a nice pattern, attempting to identify which gamma_infinity translate we have on our hands.  It will be super inefficient.

def spidercrawl(C):
    #C is an element of cycles, consisting of A_a, A_c, and gamma_3
    #returns the pair A_b, gamma_1
    
    found = 0
    gamm1Pb = C[0][1]^(-1)*C[2][1]*C[1][1]*Pinf
    s = gamm1Pb[2][0]
    if not(s == 0):
        gamm1Pb = gamm1Pb*s^(-1)
    else:
        found = 1
        return [['Id',Id],['Id',Id]]
    

    d = depth(gamm1Pb)
    levdpoints = []
    levdmats = []
    if d == 1:
        levdpoints = [points[0]]
        levdmats = [matrices[0]]
    if d == 2:
        levdpoints = [points[1],points[2]]
        levdmats = [matrices[1],matrices[2]]
    if d == 4:
        levdpoints = [points[3],points[4],points[5],points[6],points[7]]
        levdmats = [matrices[3],matrices[4],matrices[5],matrices[6],matrices[7]]
    if d == 7:
        levdpoints = [points[8],points[9],points[10],points[11],points[12],points[13]]
        levdmats = [matrices[8],matrices[9],matrices[10],matrices[11],matrices[12],matrices[13]]
    if len(levdpoints) == 0:
        print 'error: bad depth.'
        return []
    
    for p in levdpoints:
        if gamm1Pb == p[1]:
            found = 1
            return [ levdmats[levdpoints.index(p)], ['Id',Id]]

    W = matrices_gamma_infinity
    while found == 0:
        next = []
        for w in W:
            for p in levdpoints:
                candidate = w[1]*p[1]
                s = candidate[2][0]
                if not(s == 1):
                    candidate = candidate*s^(-1)
                if candidate == gamm1Pb:
                    found = 1
                    return [levdmats[levdpoints.index(p)],w]
            for m in matrices_gamma_infinity:
                next.append([m[0]+'*'+w[0],m[1]*w[1] ])
        W = next



In [12]:
def findRelation(C):
    
    B = spidercrawl(C)
    gamma = C[1][1]^-1*C[2][1]^-1*C[0][1]*B[1][1]*B[0][1]
    found = 0
    
    if gamma == Id:
        found = 1
        return '('+C[1][0]+')^-1*('+C[2][0]+')^-1*('+C[0][0]+')*('+B[1][0]+')*('+B[0][0]+') == Id'
    if gamma == -Id:
        found = 1
        return '('+C[1][0]+')^-1*('+C[2][0]+')^-1*('+C[0][0]+')*('+B[1][0]+')*('+B[0][0]+') == -Id'
    
    W = matrices_gamma_infinity
    while found == 0:
        next = []
        for w in W:
            if gamma == w[1]:
                found = 1
                return '('+C[1][0]+')^-1*('+C[2][0]+')^-1*('+C[0][0]+')*('+B[1][0]+')*('+B[0][0]+') == '+w[0]
            if gamma == -w[1]:
                found = 1
                return '('+C[1][0]+')^-1*('+C[2][0]+')^-1*('+C[0][0]+')*('+B[1][0]+')*('+B[0][0]+') == -'+w[0]
            for m in matrices_gamma_infinity:
                next.append([m[0]+'*'+w[0],m[1]*w[1] ])
        W = next
        
findRelation(cycles[0])

'(I0)^-1*(Id)^-1*(I0)*(Id)*(Id) == Id'

In [13]:
for C in cycles:
    print(findRelation(C))

(I0)^-1*(Id)^-1*(I0)*(Id)*(Id) == Id
(A22)^-1*(Id)^-1*(I0)*(T1^-1)*(A43) == T1^-1


(A42)^-1*(Id)^-1*(I0)*(Tv^-1*T1^-1)*(A21) == T1*Ttau*T1*R
(A43)^-1*(Id)^-1*(I0)*(T1*Ttau)*(I0) == -T1*Ttau
(A71)^-1*(Id)^-1*(I0)*(Tv^-1)*(I0) == -R
(A72)^-1*(Id)^-1*(I0)*(Tv^-1)*(A41) == -Id
(A21)^-1*(Id)^-1*(A21)*(Id)*(Id) == Id


(A41)^-1*(Id)^-1*(A21)*(R*T1^-1*Ttau^-1)*(A44) == Tv*Ttau^-1


(A44)^-1*(Id)^-1*(A21)*(Tv^-1*T1)*(A21) == -T1*T1*Ttau*R
(A45)^-1*(Id)^-1*(A21)*(T1)*(I0) == -Ttau*R


(A74)^-1*(Id)^-1*(A21)*(R*T1^-1*Ttau^-1)*(A45) == -Ttau^-1*T1
(A75)^-1*(Id)^-1*(A21)*(Id)*(I0) == -T1*R
(A76)^-1*(Id)^-1*(A21)*(Ttau^-1*T1*T1)*(A21) == Tv*Ttau^-1*T1^-1
(I0)^-1*(Id)^-1*(A22)*(T1^-1)*(A43) == T1^-1
(A22)^-1*(Id)^-1*(A22)*(Id)*(Id) == Id

...

(I0)^-1*(Tv^-1*T1*Ttau^-1*R)^-1*(I0)*(Ttau^-1*T1)*(I0) == -Ttau^-1*T1
(I0)^-1*(T1^2)^-1*(A21)*(Tv^-1*T1*T1)*(A72) == T1^-1


(I0)^-1*(T1^2)^-1*(A22)*(Tv^-1*T1^-1*T1^-1)*(A21) == -Tv*T1^-1*T1^-1*T1^-1


(I0)^-1*(T1^2)^-1*(A45)*(Ttau*T1^-1*Ttau*R)*(A44) == R*T1*Ttau^-1


(I0)^-1*(T1^2*R)^-1*(A21)*(Tv^-1*T1*T1)*(A72) == R*T1^-1


(I0)^-1*(T1^2*R)^-1*(A22)*(Tv^-1*T1^-1*T1^-1)*(A21) == -T1*R*T1^-1*T1^-1
